In [21]:
%run 1_Function_Master.ipynb
%run 2_Parameterization.ipynb

# Testing Individual Solution

In [22]:
''' Reading in stored solution. Just one single solution. '''
%store -r sol

In [23]:
''' Creating dataframe including sources vs. sinks for each state. '''
dfs = svs(sol, param)

## Plotting individual or combined sources and sinks for given state variable

In [24]:
""" Plotting SVS for specified state """

fig, axs = plt.subplots(3, 1, figsize = (10, 8))

labels = ['N_n', 'N_r', 'P_U', 'P_I', 'Z', 'V_I', 'V_E']
colors = ["darkviolet", "blue", "mediumaquamarine", "yellowgreen", "darkgreen", "red", "darkred"]

# Plotting states evolution
for i, df in enumerate(dfs):
    axs[0].plot(df["Time (Days)"], df[labels[i]], color=colors[i], label=labels[i])
    axs[1].plot(df["Time (Days)"], df[labels[i]], color=colors[i], label=labels[i])
    
""" assign index to 'state' accourding to label order """
state = 3 # Infected Phytoplankton
time_span = dfs[state]["Time (Days)"]

# Combined Sources and Sinks
# axs[2].plot(time_span, dfs[state]["Sources"] / dfs[state][labels[state]], color='black', label="Sources")
# axs[2].plot(time_span, dfs[state]["Sinks"] / dfs[state][labels[state]], '--', color='black', label="Sinks")
    
# Individual transfer terms
for column in dfs[state].columns[2:-3]:
    if '--' in column: 
        axs[2].plot(time_span, dfs[state][column] / dfs[state][labels[state]], '--', label=column[:-3])
    else: 
        axs[2].plot(time_span, dfs[state][column] / dfs[state][labels[state]], label=column)
    
# Axs specific settings
axs[0].set_title("System Dynamics - Linear Scaling")
axs[0].set_ylabel(r'Nutrients ($mmol/m^3$)')

axs[1].set_title("System Dynamics - Log Scaling")
axs[1].set_ylabel(r'Nutrients ($mmol/m^3$)')
axs[1].set_yscale('log')
axs[1].set_ylim(1e-12, 20)

axs[2].set_title(f"Normalized {labels[state]} Sources and Sinks")
axs[2].set_ylabel('% per day')
axs[2].set_ylim(-.01, 1)

# Broad Axs settings   
for ax in axs:
    ax.set_xlabel('Time (Day)')
    ax.set_xlim(-1, 200)
    ax.grid(which='both', alpha=.3)
    ax.legend(loc=(1.01, 0), framealpha=1)
    
plt.tight_layout()
plt.show()

# Pulling Large Solution Set

In [25]:
''' Pulling multiple solutions. '''
%store -r sols

## Creating DataFrames

In [26]:
'''Creating a data frame for each state variable that holds the 
values of each transfer term at the end of the solution time span.

The values are normalized later when plotted.'''

rate_dfs = [pd.DataFrame()] * 7

for z0, sol in sols:
    
    dfs = svs(sol, param)
    
    for i, df in enumerate(dfs):
        last_row = df.tail(1)
        last_row.insert(0, 'Total N', sum(z0), True)

        rate_dfs[i] = pd.concat([rate_dfs[i], last_row], ignore_index=True)

# Drop unecessary columns (Time (days), sources, sinks, dSdt) 
for df in rate_dfs:
    df.drop(df.columns[[1, -1, -2, -3]], axis=1, inplace=True)

In [27]:
''' Example of INfected Phytoplankton transfer rates at late time. '''
rate_dfs[3].div(rate_dfs[3]['P_I'], axis=0).tail()

,Total N,P_I,Growth,Adsorption,Grazing --,Mortality --,NT Recycling --,Lysis --,Entrainment --
195,10.657161,1.0,1.388333,0.0,1.239342,0.05,0.052101,0.046891,0.0
196,11.283245,1.0,1.389182,0.0,1.240483,0.05,0.051947,0.046752,0.0
197,11.946921,1.0,1.389957,0.0,1.241525,0.05,0.051806,0.046626,0.0
198,12.650395,1.0,1.390666,0.0,1.242477,0.05,0.051678,0.046510,0.0
199,13.396013,1.0,1.391315,0.0,1.243350,0.05,0.051561,0.046405,0.0


## Defining colors for plotting rates

In [28]:
term_colors = {}

for df in rate_dfs:
    
    for col in df.columns:
        if '--' in col:
            col = col[:-3]
            
        if col not in term_colors.keys():
            term_colors[col] = ''

remove = ('Total N', 'N_n', 'N_r', 'P_U', 'P_I', 'Z', 'V_I', 'V_E')

for key in remove:
    term_colors.pop(key, None)

colors = ['darkred', 'r', 'orangered', 'sienna', 'darkorange', 
          'goldenrod', 'olive', 'darkgreen', 'mediumaquamarine', 'teal', 
          'steelblue', 'darkblue', 'b', 'blueviolet', 'purple']

for i, key in enumerate(term_colors.keys()):
    term_colors[key] = colors[i]

## Plotting Normalized SVS

In [29]:
fig, axs = plt.subplots(6, 2, figsize=(14, 24))

labels = ['N_r', 'P_U', 'P_I', 'Z', 'V_I', 'V_E']

# Loop through state rate dfs
for i, df in enumerate(rate_dfs[1:]):

    for column in df.columns[2:]:
    
        if '--' in column: 
            axs[i][0].scatter(df['Total N'], df[column] / df.iloc[:, 1], marker='v', s=.5, alpha=.75, color=term_colors[column[:-3]], label=column[:-3])
            axs[i][1].scatter(df['Total N'], df[column] / df.iloc[:, 1], marker='v', s=.5, alpha=.75, color=term_colors[column[:-3]], label=column[:-3])

        else: 
            axs[i][0].scatter(df['Total N'], df[column] / df.iloc[:, 1], marker='^', s=.5, alpha=.75, color=term_colors[column], label=column)
            axs[i][1].scatter(df['Total N'], df[column] / df.iloc[:, 1], marker='^', s=.5, alpha=.75, color=term_colors[column], label=column)
            
    axs[i][0].set_title(fr'Normalized ${labels[i]}$ SvS - Linear Scale')
    # axs[i][0].set_xlim(-1, 21)
    
    axs[i][1].set_title(fr'Normalized ${labels[i]}$ SvS - Log Scale')
    # axs[i][1].set_xscale('log')
    # axs[i][1].set_xlim(1e-4, 4e1)
    axs[i][1].legend(markerscale=7.5, loc=(1.025, 0))
    
# Tuning Pu limits  
# axs[1][0].set_ylim(-.01, .2)
# axs[1][1].set_ylim(-.01, .2)
axs[0][1].set_xscale('log')
axs[1][1].set_xscale('log')
axs[2][1].set_xscale('log')
axs[3][1].set_xscale('log')
axs[4][1].set_xscale('log')
axs[5][1].set_xscale('log')

axs = axs.reshape(-1)

for ax in axs:
    ax.grid(which='both', alpha=.3)
    ax.set_ylabel(r'Rate (day$^{-1}$)')
    ax.set_xlabel(r'Total N (mmol m$^{-3}$)')

plt.tight_layout()
plt.show()